In [7]:
import duckdb
import polars as pl

def load_database_to_polars(db_path, table_name=None):
    """
    Load DuckDB database table(s) into Polars DataFrame(s)
    
    Parameters:
    db_path (str): Path to the DuckDB database file
    table_name (str, optional): Specific table to load. If None, loads all tables
    
    Returns:
    dict: Dictionary of table_name: polars.DataFrame pairs (if table_name is None)
    polars.DataFrame: Single DataFrame (if table_name is specified)
    """
    with duckdb.connect(db_path) as conn:
        if table_name is None:
            # Get all table names
            tables_query = """
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'main'
            """
            tables = [row[0] for row in conn.execute(tables_query).fetchall()]
            
            # Load each table into a Polars DataFrame
            dataframes = {}
            for table in tables:
                # Convert DuckDB table to Pandas first (direct Polars conversion not available)
                pandas_df = conn.execute(f"SELECT * FROM {table}").df()
                # Convert Pandas DataFrame to Polars
                dataframes[table] = pl.from_pandas(pandas_df)
            return dataframes
        else:
            # Load single specified table
            pandas_df = conn.execute(f"SELECT * FROM {table_name}").df()
            return pl.from_pandas(pandas_df)

# Usage examples:
# Load all tables
# dfs = load_database_to_polars('data/skc.db')

# Load specific table
df = load_database_to_polars('data/skc.db', 'marc_records')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
df

id,titul,autorstvo,autorstvo_kod
str,str,str,str
"""960915d19922005be zu p ||||u|…","""'30-'50""",null,null
"""960915u19uu1981fr qu p ||||u|…","""R. Rhumatologie""",null,null
"""960915c19869999fr uu p ||||u|…","""01 INFORMATIQUE""",null,null
"""960915d19922001gw uu p ||||u|…","""112""",null,null
"""960915d19742001gw zu p ||||u|…","""3 R. Rohre, Rohrleitungsbau, R…",null,null
…,…,…,…
"""250117s1986 xr |…","""jz8000066""","""Bezděk, František,""","""František Bezděk :"""
"""241204s2024 xo ||||| |||| 0…","""xx0107910""","""Ondriska, František,""","""Parazitológia :"""
"""250121s2023 xr a c 6 0…","""osa20191025252""","""Shuky,""","""Rytíři."""


In [11]:
df.filter(pl.col("autorstvo").str.contains("Kašpárek, Michal"))

id,titul,autorstvo,autorstvo_kod
str,str,str,str
"""060624s2006 xr ||||| ||||||…","""Použitelnost a přístupnost web…","""Kašpárek, Michal""",null
"""180516s2018 xr g 0…","""Hry bez hranic /""","""Kašpárek, Michal,""","""jx20100301005"""
"""200115s2019 xr nnnng …","""Hry bez hranic /""","""Kašpárek, Michal,""","""jx20100301005"""
"""220209s2020 xr g …","""Hry bez hranic /""","""Kašpárek, Michal,""","""jx20100301005"""
"""230925s2023 xr g 0…","""Fosilie /""","""Kašpárek, Michal,""","""jx20100301005"""


In [27]:
df_s = df.sample(1000)

In [37]:
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno

In [63]:
zahranici = df.filter(pl.col('id').str.slice(15,2) != 'xr').with_columns(pl.col('id').map_elements(najdi_rok, return_dtype=int).alias('rok')).drop_nulls(subset=['autorstvo']).with_columns(pl.col("autorstvo").map_elements(hezke_jmeno).alias("jmeno"))

C:\Users\micha\AppData\Local\Temp\ipykernel_12044\1620344683.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  zahranici = df.filter(pl.col('id').str.slice(15,2) != 'xr').with_columns(pl.col('id').map_elements(najdi_rok, return_dtype=int).alias('rok')).drop_nulls(subset=['autorstvo']).with_columns(pl.col("autorstvo").map_elements(hezke_jmeno).alias("jmeno"))


In [64]:
zahranici.filter(pl.col('jmeno').str.contains('Škvorecký')).filter(pl.col('rok').is_in(1985))

id,titul,autorstvo,autorstvo_kod,rok,jmeno
str,str,str,str,i64,str
"""980115s1985 xxu f |…","""The bass saxophone :""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""
"""911103t19851984xxu g 0…","""The engineer of human souls /""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""
"""210824s1985 xxca g 0…","""Ze života české společnosti /""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""
"""960412s1985 xxc||||| ||||||…","""The swell season :""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""
"""081014s1985 xxu |…","""Miss Silver's past /""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""
"""960116s1985 xxk 0…","""The engineer of human souls :""","""Škvorecký, Josef,""","""jk01130413""",1985,"""Josef Škvorecký"""


In [67]:
znormalizovani_top_domaci_razeni = ['Josef Škvorecký',
 'Ivan Klíma',
 'Václav Havel',
 'Felix Háj',
 'Tomáš Garrigue Masaryk',
 'Karel Kaplan',
 'Jakub Deml',
 'Vladimír Škutina',
 'Anna Sedlmayerová',
 'Václav Černý',
 'Karel Šiktanc']

In [73]:
import os

In [75]:
zahranici.filter(pl.col('jmeno').is_in(znormalizovani_top_domaci_razeni)).write_parquet(os.path.join('data','zahranicni_knihy_10_zakazanych_autoru.parquet'))